# Step 3: Fully Expandable Problem

This notebook covers the code for our expandable solution with no noise mitigation. The expandable QArray we used for step 2 of the toy problem code now has 16 qubits, which can be increased further, with the same gate structure as before that expands across the entire qubit register instead of just 4 qubits.

In [16]:
from classiq import *
from classiq.execution import ExecutionPreferences
from math import pi

In [17]:
TROTTER_STEPS = 4
THETA_H = 0
SHOTS = 100
QUBITS = 16

In [24]:
@qfunc
def trotter_step(arr: QArray[QBit]):
    # apply a rotation over the X axis on all qubits with an angle of theta_h
    apply_to_all(lambda q: RX(THETA_H, q), arr)


    apply_to_all(lambda q: invert(lambda: S(q)), arr)

    for i in range(0, QUBITS-1, 2):
        within_apply(compute=lambda: RY(pi/2, arr[i+1]), action=lambda: CX(arr[i], arr[i+1]))


    apply_to_all(lambda q: invert(lambda: S(q)), arr)

    for i in range(1, QUBITS, 2):
        if i == QUBITS-1:
            within_apply(compute=lambda: RY(pi/2, arr[i]), action=lambda: CX(arr[0], arr[i]))
        else:
            within_apply(compute=lambda: RY(pi/2, arr[i+1]), action=lambda: CX(arr[i], arr[i+1]))



# Trotterized time evolution function for the multi-qubit QArray
@qfunc
def trotterized_time_evolution(arr: QArray[QBit]):
    for _ in range(TROTTER_STEPS):
      trotter_step(arr)

@qfunc
def main(expectation_value: Output[QBit]):
    # initialize the qubit register
    arr = QArray("reg")
    allocate(QUBITS, arr)


    # setup for measurement of the expectation value
    allocate(1, expectation_value)

    within_apply(lambda: hadamard_transform(expectation_value), lambda: control(expectation_value, lambda: trotterized_time_evolution(arr)))


In [26]:
# Gets the counts of measuring the qubits in their respective states
def synthesize_execute(shots: int):
    quantum_model = set_execution_preferences(
    create_model(main),
    ExecutionPreferences(num_shots=shots),
    )

    quantum_program = synthesize(quantum_model)
    job = execute(quantum_program)
    results = job.result()[0].value.parsed_counts

    return results

# Evaluates the real part of expectation value
def evaluate_real_part(shots):
    print(f"Executing with {shots} shots...")
    result = synthesize_execute(shots)

    if result[0].state["expectation_value"] == 0:
        prob = result[0].shots / shots
    else:
        prob = result[1].shots / shots

    print(f"Measured 0 with probability of {prob*100:.3f}%")

    return 2*prob - 1

One important point to note is that `THETA_H` being set to zero basically means that the transverse magnetic field was not included in the simulation up to this point, because in the second page of the paper, the third average magnetization plot had multiple data points for an average magnetization of around 1.

In [27]:
value = evaluate_real_part(SHOTS)
print("Expectation value:", value)

Executing with 100 shots...
Measured 0 with probability of 100.000%
Expectation value: 1.0


Show diagram of the quantum circuit

In [21]:
qprog = synthesize(create_model(main))
show(qprog)

Opening: https://platform.classiq.io/circuit/93e0e0b0-cf86-4c36-bbc2-437ca4d7695d?version=0.43.3


Opening in existing browser session.
